In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

function printer(x::Any, verbose="")
    if verbose != ""
        println(verbose)
    end
    show(stdout, "text/plain", x)
    println("")
end

In [ ]:
using CSV, DataFrames, Plots, Statistics, StatsBase, Random, Distributions, LinearAlgebra
gr();

In [ ]:
mutable struct Agent
    v_::Float64
    ω_::Float64
    function Agent(v::Float64, ω::Float64)
        new(v, ω)
    end
end
function decision(agent::Agent, observation=nothing)
    return agent.v_, agent.ω_
end
mutable struct IdealRobot
    pose_::Array{Float64, 1}
    agent_::Agent
    radius_::Float64
    color_::String
    poses_::Array{Array{Float64, 1}, 1}
    function IdealRobot(pose::AbstractArray{Float64, 1}, agent::Agent, radius=0.2, color="blue")
        new([pose[1], pose[2], pose[3]], agent, radius, color, [copy(pose)])
    end
end
function draw(robot::IdealRobot, p)
    p = scatter!([robot.pose_[1]], [robot.pose_[2]], markersize=robot.radius_ * 100, color=robot.color_, markeralpha=0.5, legend=nothing, aspect_ratio=:equal)
    θ = robot.pose_[3]
    p = quiver!([robot.pose_[1]], [robot.pose_[2]], quiver=([robot.radius_ * cos(θ) * 5], [robot.radius_ * sin(θ) * 5]), color="black")
    x_his = [pose[1] for pose in robot.poses_]
    y_his = [pose[2] for pose in robot.poses_]
    p = plot!(x_his, y_his, color="black", lw=0.5)
end
function state_transition(robot::IdealRobot, dt::Float64)
    v = robot.agent_.v_
    ω = robot.agent_.ω_
    θ = robot.pose_[3]
    new_pose = [0.0, 0.0, 0.0]
    if abs(ω) < 1e-10
        dpose = [v * cos(θ), v * sin(θ), ω] * dt
        new_pose = robot.pose_ .+ dpose
    else
        dpose = [v / ω * ( sin(θ + ω * dt) - sin(θ)), v / ω * ( -cos(θ + ω * dt) + cos(θ)), ω * dt]
        new_pose = robot.pose_ .+ dpose
    end
    push!(robot.poses_, robot.pose_)
    robot.pose_ = copy(new_pose)
    return new_pose
end
mutable struct World
    objects_::Array{Any, 1}
    xlim_::Array{Float64, 1}
    ylim_::Array{Float64, 1}
    debug_::Bool
    function World(xlim::AbstractArray{Float64, 1}, ylim::AbstractArray{Float64, 1}, debug=false)
        new(Array{Any, 1}[], [xlim[1], xlim[2]], [ylim[1], ylim[2]], debug)
    end
end

function Base.push!(world::World, robot::IdealRobot)
    push!(world.objects_, robot)
end

function draw(world::World, annota=nothing)
    p = plot(aspect_ratio=:equal, xlim=world.xlim_, ylim=world.ylim_)
    xpos = (world.xlim_[1] + world.xlim_[2]) / 2.0
    ypos = world.ylim_[2] - 0.1 * (world.ylim_[2] - world.ylim_[1])
    if annota != nothing
        p = annotate!(xpos, ypos, annota)
    end
    for obj = world.objects_
        draw(obj, p)
    end
    return p
end

In [ ]:
straight_agent = Agent(0.15, 0.0)
circling_agent = Agent(0.2, 10.0 / 180.0 * pi)
robot1 = IdealRobot([2.0, 3.0, pi/6], straight_agent, 0.1, "blue",)
robot2 = IdealRobot([-2.0, -1.0, pi*5/6], circling_agent, 0.08, "orange")
xlim = [-5.5, 10]
ylim = [-5.5, 10]
world = World(xlim, ylim)
push!(world, robot1)
push!(world, robot2)
dt = 0.1
anim = @animate for i = 1:200
    t = dt * i
    annota = "t = $(round(t, sigdigits=3))[s]"
    p = draw(world, annota)
    plot(p)
    state_transition(robot1, dt)
    state_transition(robot2, dt)
end
gif(anim, "images/ch3_robot05.gif", fps=10)